In [1]:
# Include so results on different machines are (should be) the same.
import numpy as np
from numpy.random import seed
seed(1)

from tensorflow import set_random_seed
set_random_seed(2)

import random as rn
rn.seed(3)

In [2]:
!jupyter nbconvert --to script Keras_BagnallCharacter_SimpleRNN.ipynb

[NbConvertApp] Converting notebook Keras_BagnallCharacter_SimpleRNN.ipynb to script
[NbConvertApp] Writing 14529 bytes to Keras_BagnallCharacter_SimpleRNN.py


In [60]:
import glob, os, json, re, unicodedata
from bs4 import BeautifulSoup

load_verbose = 0
loaded_labels = []
loaded_text = []
presidents = [
    "Barack Obama",
    "Donald J. Trump",
    "Dwight D. Eisenhower",
    "Franklin D. Roosevelt",
    "George Bush",
    "George W. Bush",
    "Gerald R. Ford",
    "Harry S. Truman",
    "Herbert Hoover",
    "Jimmy Carter",
    "John F. Kennedy",
    "Lyndon B. Johnson",
    "Richard Nixon",
    "Ronald Reagan",
    "William J. Clinton"
]

labels = {}
for idx, name in enumerate(presidents):
    labels[name] = idx

# load raw text files straight in, no parsing
file_to_label = {
    "Obama": "Barack Obama",
    "Trump": "Donald J. Trump",
    "Eisenhower": "Dwight D. Eisenhower",
    "Roosevelt": "Franklin D. Roosevelt",
    "Bush": "George Bush",
    "WBush": "George W. Bush",
    "Ford": "Gerald R. Ford",
    "Truman": "Harry S. Truman",
    "Hoover": "Herbert Hoover",
    "Carter": "Jimmy Carter",
    "Kennedy": "John F. Kennedy",
    "Johnson": "Lyndon B. Johnson",
    "Nixon": "Richard Nixon",
    "Reagan": "Ronald Reagan",
    "Clinton": "William J. Clinton"
}

directory = "../data/processed/"
for filename in glob.glob(os.path.join(directory, '*.txt')):
    arr = filename.replace(directory,'').split("_")
    if any(prefix in arr[0] for prefix in file_to_label.keys()):
        loaded_labels = loaded_labels + [labels[file_to_label[arr[0]]]]
        raw = open(filename).read().decode("UTF-8").encode("ascii","ignore")
        loaded_text = loaded_text + [raw] 


print "Loaded", len(loaded_text), "speeches for", len(set(loaded_labels)), "presidents."
# processed2 now contains files generated from unprocessed
directory = "../data/processed3/"
for filename in glob.glob(os.path.join(directory, '*.txt')):
    arr = filename.replace(directory,'').split("_")
    if any(prefix in arr[0] for prefix in file_to_label.keys()):
        loaded_labels = loaded_labels + [labels[file_to_label[arr[0]]]]
        raw = open(filename).read().decode("UTF-8").encode("ascii","ignore")
        loaded_text = loaded_text + [raw] 

print "Loaded", len(loaded_text), "speeches for", len(set(loaded_labels)), "presidents."

Loaded 47 speeches for 14 presidents.
Loaded 1764 speeches for 15 presidents.


In [21]:
# from nltk import word_tokenize
# from collections import Counter
# from nltk.tag.perceptron import PerceptronTagger

# tagger = PerceptronTagger()

# assess_text = word_tokenize(" ".join(loaded_text))

# tagged_sent = tagger.tag(assess_text) 
# propernouns = [word for word, pos in tagged_sent if pos == 'NNP']

# print Counter(propernouns).most_common()[:-1000-1:-1]

[('APPALACHIAN', 1), ('Carmelo', 1), ('HARRY', 1), ('Warburg', 1), ('Fourteenth', 1), ('HOLMES', 1), ("'18", 1), ('Barranquilla', 1), ('Gates/Iraq', 1), ('Proposition', 1), ('Mario', 1), ('Rashid', 1), ('STIMULUS', 1), ('HER', 1), ('Bailout', 1), ('Ricky', 1), ('Writer', 1), ('Goldner', 1), ('Departments-Army', 1), ('INCURSION', 1), ('Ter', 1), ('TORRIJOS', 1), ('Quezon', 1), ('Kitona', 1), ('September-so', 1), ('Sills', 1), ('Anastasio', 1), ('Penultimate', 1), ('EFPs', 1), ('Silly', 1), ('MANAGUA', 1), ('Except', 1), ('Amtorg', 1), ('Shipman', 1), ('Condition', 1), ('Vern', 1), ('Ordinance', 1), ('Anywhere', 1), ('Rochambeau', 1), ('Upon', 1), ('Warlick', 1), ('Seamen', 1), ('Spears', 1), ('Along', 1), ('Clashes', 1), ('Marseilles', 1), ('Group/U.S', 1), ('RADIATION', 1), ('DEPUTY', 1), ('Portrait', 1), ('Halleck', 1), ('Appropriations-I', 1), ('Brownsville', 1), ('Antiapartheid', 1), ('Macs', 1), ('Dollars', 1), ('ATTICA', 1), ('Population', 1), ('States-Mexico', 1), ('Process/Iran'

In [61]:
#
# Bagnall 2015 text pre-processing
#
from string import maketrans
import re

chars_to_replace = "[]%!()>=*&_}+"
sub_chars = len(chars_to_replace) * " "
trantab = maketrans(chars_to_replace, sub_chars)
for x in range(0,len(loaded_text)):
    # "Various rare characters that seemed largely equivalent are mapped together..."
    loaded_text[x] = re.sub('`', '\'', loaded_text[x])
    loaded_text[x] = re.sub('--', '-', loaded_text[x])
    loaded_text[x] = re.sub('\n\n', '\n', loaded_text[x])
    # "...all digits in all languages are mapped to 7"
    loaded_text[x] = re.sub('[0-9]+', '7', loaded_text[x])
    # "...any character with a frequency lower than 1 in 10,000 is discarded."
    loaded_text[x] = loaded_text[x].translate(trantab)
    # "Runs of whitespace are collapsed into a single space."
    loaded_text[x] = re.sub(' +', ' ', loaded_text[x])
    
    # REPLACE WORD IN ALL CAPS with <t> as a test of topic removal
    #loaded_text[x] = re.sub('[A-Z]{2,}','<t>', loaded_text[x])

print "Replacements complete."

Replacements complete.


In [62]:
# Have a look at a scrubbed text excerpt
print loaded_text[200][:750]

RESCUE MISSION FOR AMERICAN HOSTAGES IN IRAN Before answering questions this evening, I would like to say a few words about the rescue mission in Iran. I share the disappointment of the American people that this rescue mission was not successful, and I also share the grief of our Nation because we had Americans who were casualties in this effort to seek freedom for their fellow citizens who have been held hostage for so long. But I also share a deep pride in the commitment and courage and the integrity and the competence and the determination of those who went on this mission. They were prepared to do their duty, and they did their duty. I can think of no higher compliment for a Commander in Chief to pay to brave men. It was my responsibili


In [63]:
#
# Join all speeches into one massive per president
#  for later processing
#
import numpy as np
from scipy import stats
from operator import itemgetter
from collections import defaultdict

compressed_text = [None]*(len(labels))
for key, value in sorted(labels.iteritems()):
    compressed_text[value] = ""
    for idx in range(0,len(loaded_text)):
        if (loaded_labels[idx] == value):
            compressed_text[value] = compressed_text[value] + loaded_text[idx] + " "
            
print "How many characters of text per president?"
for key, value in sorted(labels.iteritems()):
    print str(value).ljust(2), ":", key.ljust(20), "\t", len(compressed_text[value])

label_min_chars = len(min(compressed_text, key=len))
print "\nMinimum number of characters per president?"
print label_min_chars

How many characters of text per president?
0  : Barack Obama         	4869126
1  : Donald J. Trump      	555750
2  : Dwight D. Eisenhower 	3085442
3  : Franklin D. Roosevelt 	1746653
4  : George Bush          	1899100
5  : George W. Bush       	1792991
6  : Gerald R. Ford       	706711
7  : Harry S. Truman      	2002254
8  : Herbert Hoover       	800912
9  : Jimmy Carter         	1290474
10 : John F. Kennedy      	1345102
11 : Lyndon B. Johnson    	2326890
12 : Richard Nixon        	990798
13 : Ronald Reagan        	1010956
14 : William J. Clinton   	1786719

Minimum number of characters per president?
555750


In [64]:
#
# Tokenize words into chars
#
from keras.preprocessing.text import Tokenizer
from keras.preprocessing import sequence

# Tokenize into characters
tokenizer = Tokenizer(char_level=True)
tokenizer.fit_on_texts(compressed_text)
tokenized_text = tokenizer.texts_to_sequences(compressed_text)

# there's an oddity in the encoding for some reason where a len+1 character occurs
unique_chars = len(tokenizer.word_counts)+1

print "Unique char count:", unique_chars
print "\nChars w/ counts:"
print sorted(((v,k) for k,v in tokenizer.word_counts.iteritems()), reverse=True)

Unique char count: 68

Chars w/ counts:
[(4725447, ' '), (2569974, 'e'), (2131220, 't'), (1627649, 'o'), (1609954, 'a'), (1476541, 'n'), (1405067, 'i'), (1193120, 's'), (1157275, 'r'), (1066798, 'h'), (755187, 'l'), (705084, 'd'), (548669, 'u'), (534278, 'c'), (478832, 'm'), (418343, 'w'), (393337, 'g'), (389784, 'f'), (382020, 'p'), (374441, 'y'), (281512, 'b'), (264600, ','), (263713, '.'), (240856, 'v'), (175656, 'I'), (151063, 'k'), (95930, "'"), (74786, 'A'), (59022, 'T'), (58392, 'S'), (44390, '-'), (40592, 'W'), (40548, '7'), (35537, 'C'), (33805, 'N'), (32811, 'x'), (27664, 'M'), (25791, 'P'), (25503, 'j'), (25399, 'B'), (23916, 'E'), (21726, 'q'), (20252, 'R'), (19956, 'O'), (19583, 'G'), (17171, 'H'), (16245, 'U'), (15855, 'D'), (13886, 'F'), (12873, 'L'), (11930, 'z'), (10681, 'Y'), (10672, '"'), (9360, ';'), (8141, '?'), (8075, 'J'), (6390, ':'), (5293, 'V'), (4737, '\n'), (4486, 'K'), (3981, '$'), (1853, '/'), (1349, 'Q'), (446, 'Z'), (421, 'X'), (6, '#'), (4, '\\')]


In [65]:
#
# Split speeches into subsequences 
#
from collections import Counter

def splits(_list, _split_size):
    output_list = []
    for idx in range(0, len(_list), _split_size):
        if (idx + _split_size) <= len(_list):
            output_list.append(_list[idx:idx + _split_size])
    return output_list

max_seq_len = 50

# create new speech/label holders
split_text = []
split_labels = []

for idx in range(0, len(tokenized_text)):
    current_label = idx
    current_speech = tokenized_text[idx]#[:label_min_chars]
    current_splits = splits(current_speech, max_seq_len)
    split_text.extend(current_splits)
    split_labels.extend([current_label] * len(current_splits))

print "Subsequence total count; subsequence label total count:", len( split_text ), len( split_labels )
print "\nTotal characters:", len( split_text ) * max_seq_len

Subsequence total count; subsequence label total count: 524192 524192

Total characters: 26209600


In [66]:
# Have a look at a few split text excerpts
print split_text[10:15]
print split_labels[10:15]

[[9, 7, 6, 17, 1, 14, 11, 4, 13, 12, 8, 1, 5, 6, 12, 1, 9, 5, 17, 7, 6, 17, 1, 8, 3, 4, 9, 15, 8, 23, 1, 28, 3, 1, 3, 10, 2, 8, 2, 1, 15, 4, 15, 2, 6, 3, 8, 22, 1, 28], [15, 2, 9, 7, 14, 5, 1, 10, 5, 8, 1, 14, 5, 9, 9, 7, 2, 12, 1, 4, 6, 1, 6, 4, 3, 1, 8, 7, 15, 19, 11, 20, 1, 21, 2, 14, 5, 13, 8, 2, 1, 4, 18, 1, 3, 10, 2, 1, 8, 26], [7, 11, 11, 1, 4, 9, 1, 24, 7, 8, 7, 4, 6, 1, 4, 18, 1, 3, 10, 4, 8, 2, 1, 7, 6, 1, 10, 7, 17, 10, 1, 4, 18, 18, 7, 14, 2, 22, 1, 21, 13, 3, 1, 21, 2, 14, 5, 13, 8, 2], [1, 16, 2, 1, 3, 10, 2, 1, 19, 2, 4, 19, 11, 2, 1, 10, 5, 24, 2, 1, 9, 2, 15, 5, 7, 6, 2, 12, 1, 18, 5, 7, 3, 10, 18, 13, 11, 1, 3, 4, 1, 3, 10, 2, 1, 7, 12, 2, 5, 11], [8, 1, 4, 18, 1, 4, 13, 9, 1, 18, 4, 9, 2, 21, 2, 5, 9, 8, 1, 5, 6, 12, 1, 3, 9, 13, 2, 1, 3, 4, 1, 4, 13, 9, 1, 18, 4, 13, 6, 12, 7, 6, 17, 1, 12, 4, 14, 13, 15, 2]]
[0, 0, 0, 0, 0]


In [67]:
#
# split amongst speaker samples, not the whole population of samples; shuffle if requested
#
import sklearn.utils

def split_test_train(input_text, input_labels, labels, train_pct=0.8, shuffle_p=True):
    train_text = []
    train_labels = []
    test_text = []
    test_labels = []

    for key, value in sorted(labels.iteritems()):
        # grab all values of a specific label
        subset_text = list(itemgetter(*[idx for idx, label in enumerate(input_labels) if label == value ])(input_text))
        subset_labels = list(itemgetter(*[idx for idx, label in enumerate(input_labels) if label == value ])(input_labels))

        cut_pos = int(train_pct * len(subset_text))
        train_text = train_text + subset_text[:cut_pos]
        train_labels = train_labels + subset_labels[:cut_pos]
        test_text = test_text + subset_text[cut_pos:]
        test_labels = test_labels + subset_labels[cut_pos:]

    if shuffle_p:
        test_text, test_labels = sklearn.utils.shuffle(test_text, test_labels)
        train_text, train_labels = sklearn.utils.shuffle(train_text, train_labels)

    return train_text, train_labels, test_text, test_labels

In [68]:
#
# Prep test/train
#
from sklearn.preprocessing import OneHotEncoder
from sklearn.utils import class_weight

# split data smartly
train_X, train_y, test_X, test_y = split_test_train(split_text, split_labels, 
                                                    labels, train_pct=0.8, shuffle_p=False)
print "Sample splits:\n Test = ", len(train_X), "\n Train = ", len(test_X)##

# compute class weights to account for imbalanced classes
y_weights = (class_weight.compute_class_weight('balanced', np.unique(train_y), train_y)).tolist()
y_weights = dict(zip(sorted(labels.values()), y_weights))
print "\nClass weights:\n", y_weights

Sample splits:
 Test =  419349 
 Train =  104843

Class weights:
{0: 0.3588550157242796, 1: 3.144017094017094, 2: 0.5663128468986752, 3: 1.000379302941387, 4: 0.9200789863419451, 5: 0.9745389897863144, 6: 2.472503758733528, 7: 0.8726620052441004, 8: 2.181723115342594, 9: 1.3540272194507676, 10: 1.2990381487849079, 11: 0.7509360982030138, 12: 1.763600807469089, 13: 1.7283833075734159, 14: 0.9779480183300101}


In [69]:
# Have a look at a few of the split text excerpts; 
# likely the same classes based on the split point
print train_X[10:15]
print train_y[10:15]

[[9, 7, 6, 17, 1, 14, 11, 4, 13, 12, 8, 1, 5, 6, 12, 1, 9, 5, 17, 7, 6, 17, 1, 8, 3, 4, 9, 15, 8, 23, 1, 28, 3, 1, 3, 10, 2, 8, 2, 1, 15, 4, 15, 2, 6, 3, 8, 22, 1, 28], [15, 2, 9, 7, 14, 5, 1, 10, 5, 8, 1, 14, 5, 9, 9, 7, 2, 12, 1, 4, 6, 1, 6, 4, 3, 1, 8, 7, 15, 19, 11, 20, 1, 21, 2, 14, 5, 13, 8, 2, 1, 4, 18, 1, 3, 10, 2, 1, 8, 26], [7, 11, 11, 1, 4, 9, 1, 24, 7, 8, 7, 4, 6, 1, 4, 18, 1, 3, 10, 4, 8, 2, 1, 7, 6, 1, 10, 7, 17, 10, 1, 4, 18, 18, 7, 14, 2, 22, 1, 21, 13, 3, 1, 21, 2, 14, 5, 13, 8, 2], [1, 16, 2, 1, 3, 10, 2, 1, 19, 2, 4, 19, 11, 2, 1, 10, 5, 24, 2, 1, 9, 2, 15, 5, 7, 6, 2, 12, 1, 18, 5, 7, 3, 10, 18, 13, 11, 1, 3, 4, 1, 3, 10, 2, 1, 7, 12, 2, 5, 11], [8, 1, 4, 18, 1, 4, 13, 9, 1, 18, 4, 9, 2, 21, 2, 5, 9, 8, 1, 5, 6, 12, 1, 3, 9, 13, 2, 1, 3, 4, 1, 4, 13, 9, 1, 18, 4, 13, 6, 12, 7, 6, 17, 1, 12, 4, 14, 13, 15, 2]]
[0, 0, 0, 0, 0]


In [70]:
#
# One-hot encoding classes & samples
#
from keras.utils import to_categorical

# one-hot encode classes
train_y = to_categorical(train_y)
test_y = to_categorical(test_y)

# one-hot encode samples
train_X = np.array(train_X)
orig_train_X_size=train_X.shape[0]
print "Encoding train_X with dimensions ", train_X.shape
train_X = to_categorical(train_X, num_classes=unique_chars)
print "...to ", train_X.shape
train_X = np.reshape(train_X,(orig_train_X_size,max_seq_len,unique_chars))
print "...and reshaping to ", train_X.shape

test_X = np.array(test_X)
orig_test_X_size=test_X.shape[0]
print "\nEncoding test_X with dimensions ", test_X.shape
test_X = to_categorical(test_X, num_classes=unique_chars)
print "...to ", test_X.shape
test_X = np.reshape(test_X,(orig_test_X_size,max_seq_len,unique_chars))
print "...and reshaping to ", test_X.shape

Encoding train_X with dimensions  (419349, 50)
...to  (20967450, 68)
...and reshaping to  (419349, 50, 68)

Encoding test_X with dimensions  (104843, 50)
...to  (5242150, 68)
...and reshaping to  (104843, 50, 68)


In [71]:
# Have a again look at a few of the split and encoded text excerpts; 
# both arrays should be one-hot encoded.
print train_X[10:11]
print train_y[10:11]

[[[ 0.  0.  0. ...,  0.  0.  0.]
  [ 0.  0.  0. ...,  0.  0.  0.]
  [ 0.  0.  0. ...,  0.  0.  0.]
  ..., 
  [ 0.  0.  0. ...,  0.  0.  0.]
  [ 0.  1.  0. ...,  0.  0.  0.]
  [ 0.  0.  0. ...,  0.  0.  0.]]]
[[ 1.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.]]


In [72]:
# custom activation from Bagnall 2015
#  we were never able to get this to work; either nan'ed or never converged
import tensorflow as tf
from keras.utils.generic_utils import get_custom_objects

def ReSQRT(x):
    cond = tf.less_equal(x, 0.0)
    return tf.where(cond, x * 0.0, tf.sqrt(x+1)-1)

get_custom_objects().update({'ReSQRT': ReSQRT})

Bagnall proposes that the following possible values contribute to the success of the model:

| meta-parameter                  	| typical value                      	|
|---------------------------------	|------------------------------------	|
| initial adagrad learning scale  	| 0.1, 0.14, 0.2, 0.3                	|
| initial leakage between classes 	| 1/4N to 5/N                        	|
| leakage decay (per sub-epoch)   	| 0.67 to 0.9                        	|
| hidden neurons                  	| 79, 99, 119, 139                   	|
| presynaptic noise σ             	| 0, 0.1, 0.2, 0.3, 0.5              	|
| sub-epochs                      	| 6 to 36                            	|
| text direction                  	| forward or backward                	|
| text handling                   	| sequential, concatenated, balanced 	|
| initialisation                  	| gaussian, zero                     	|

In [73]:
##
## BASELINE
##
from keras.layers import Input, Dense, SimpleRNN, Bidirectional, Dropout
from keras.callbacks import ReduceLROnPlateau, CSVLogger
from keras.optimizers import Adagrad, adam
from keras.models import Model
from keras.utils import plot_model

# define operating vars
optimizer='rmsprop'
dropout = 0.5422412690636627
activation = "relu"
batch_size = 50
units = 50
shuffle = True
epochs = 50
merge_mode = 'ave'

# define any callbacks
reduce_lr = ReduceLROnPlateau(monitor='categorical_accuracy', 
                              factor=0.5,
                              patience=1, 
                              verbose=1)
csv_logger = CSVLogger('Keras_BagnallCharacter_SimpleRNN.log')

# assemble & compile model
main_input = Input(shape=(max_seq_len,unique_chars,))
rnn = Bidirectional(SimpleRNN(units=units,
                              activation=activation),
                    merge_mode=merge_mode)(main_input)
drop = Dropout(dropout)(rnn)
main_output = Dense(len(labels),
                    activation='softmax',
                    kernel_initializer='random_uniform')(drop)
model = Model(inputs=[main_input], outputs=[main_output])

model.compile(loss='categorical_crossentropy', 
              optimizer=optimizer, 
              metrics=['categorical_accuracy'])
plot_model(model, to_file='Keras_BagnallCharacter_SimpleRNN.png', show_shapes=True, show_layer_names=True)
print(model.summary())


# train
model.fit([np.array(train_X)],
          [np.array(train_y)],
          batch_size=batch_size,
          epochs=epochs,
          shuffle=shuffle,
          class_weight = y_weights,
          callbacks=[reduce_lr, csv_logger],
          verbose=1)

model.save('Keras_BagnallCharacter_SimpleRNN.h5')
print ("Model saved.")
del model

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_2 (InputLayer)         (None, 50, 68)            0         
_________________________________________________________________
bidirectional_2 (Bidirection (None, 50)                11900     
_________________________________________________________________
dropout_2 (Dropout)          (None, 50)                0         
_________________________________________________________________
dense_2 (Dense)              (None, 15)                765       
Total params: 12,665
Trainable params: 12,665
Non-trainable params: 0
_________________________________________________________________
None
Epoch 1/50
254750/419349 [=================>............] - ETA: 143s - loss: 2.6807 - categorical_accuracy: 0.0823

KeyboardInterrupt: 

In [ ]:
# Load computed model
from keras.models import load_model
# returns a compiled model identical to the one trained
model = load_model('Keras_BagnallCharacter_SimpleRNN.h5')
print ("Model re-loaded.")

In [ ]:
from sklearn import metrics

# Evaluate performance
print "Evaluating test data..."
loss_and_metrics = model.evaluate(test_X, test_y)
print model.metrics_names
print loss_and_metrics

# Make some predictions
print "\nPredicting using test data..."
pred_y = model.predict(test_X, batch_size=batch_size, verbose=1)
pred_y_collapsed = np.argmax(pred_y, axis=1)
test_y_collapsed = np.argmax(test_y, axis=1)
print "\n\nDone prediction."

print "\nAUC = ", metrics.roc_auc_score(test_y, pred_y)

In [ ]:
# Plot confusion matrix
#   from scikit-learn examples @
#   http://scikit-learn.org/stable/auto_examples/model_selection/plot_confusion_matrix.html 
%matplotlib inline
import itertools
from sklearn.metrics import confusion_matrix
import matplotlib.pyplot as plt

def plot_confusion_matrix(cm, classes,
                          normalize=False,
                          title='Confusion matrix',
                          cmap=plt.cm.Blues):
    """
    This function prints and plots the confusion matrix.
    Normalization can be applied by setting `normalize=True`.
    """
    plt.imshow(cm, interpolation='nearest', cmap=cmap)
    plt.title(title)
    plt.colorbar()
    tick_marks = np.arange(len(classes))
    plt.xticks(tick_marks, classes, rotation=90)
    plt.yticks(tick_marks, classes)

    if normalize:
        cm = cm.astype('float') / cm.sum(axis=1)[:, np.newaxis]
        print("Normalized confusion matrix")
    else:
        print('Confusion matrix, without normalization')
        print(np.sum(cm,axis=0))

    print(cm)

    thresh = cm.max() / 2.
    for i, j in itertools.product(range(cm.shape[0]), range(cm.shape[1])):
        plt.text(j, i, cm[i, j],
                 horizontalalignment="center",
                 color="white" if cm[i, j] > thresh else "black")

    #plt.tight_layout()
    plt.ylabel('True label')
    plt.xlabel('Predicted label')

# Compute confusion matrix
cnf_matrix = confusion_matrix(test_y_collapsed, pred_y_collapsed)
np.set_printoptions(precision=2)

# Plot non-normalized confusion matrix
plt.figure(figsize=(10,10))
plot_confusion_matrix(cnf_matrix, classes=(sorted(labels, key=labels.get)),
                      title='Confusion matrix, without normalization')

#Plot normalized confusion matrix
plt.figure(figsize=(10,10))
plot_confusion_matrix(np.round(cnf_matrix,2), classes=(sorted(labels, key=labels.get)), normalize=True,
                      title='Normalized confusion matrix')

plt.show()

In [ ]:
count = 0
def reverse_map(map):
    return dict((v,k) for k,v in map.iteritems())
token_word_map = reverse_map(tokenizer.word_index)
president_map = reverse_map(labels)
def to_words(a):
    return " ".join([token_word_map[id] for id in a if id != 0])
sample_size = 20
def print_row(row):
    print "".join(col.ljust(22) for col in row)
print_row(["Predicted","Correct","Sentence"])
print_row(["----------","----------","---------------"])
sample = []
for i in range(len(test_y_collapsed)):
    if (pred_y_collapsed[i] != test_y_collapsed[i] and count < sample_size):
        sample += [[president_map[pred_y_collapsed[i]], president_map[test_y_collapsed[i]], to_words(test_X[i])]]
        count += 1
        
for row in sample:
    print_row(row)